# Text Translation and Sentiment Analysis using Transformers

## Project Overview:

The objective of this project is to analyze the sentiment of movie reviews in three different languages - English, French, and Spanish. We have been given 30 movies, 10 in each language, along with their reviews and synopses in separate CSV files named `movie_reviews_eng.csv`, `movie_reviews_fr.csv`, and `movie_reviews_sp.csv`.

- The first step of this project is to convert the French and Spanish reviews and synopses into English. This will allow us to analyze the sentiment of all reviews in the same language. We will be using pre-trained transformers from HuggingFace to achieve this task.

- Once the translations are complete, we will create a single dataframe that contains all the movies along with their reviews, synopses, and year of release in all three languages. This dataframe will be used to perform sentiment analysis on the reviews of each movie.

- Finally, we will use pretrained transformers from HuggingFace to analyze the sentiment of each review. The sentiment analysis results will be added to the dataframe. The final dataframe will have 30 rows


The output of the project will be a CSV file with a header row that includes column names such as **Title**, **Year**, **Synopsis**, **Review**, **Review Sentiment**, and **Original Language**. The **Original Language** column will indicate the language of the review and synopsis (*en/fr/sp*) before translation. The dataframe will consist of 30 rows, with each row corresponding to a movie.

In [20]:
# imports
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer
from transformers import pipeline

### Get data from `.csv` files and then preprocess data

In [21]:
# use the `pd.read_csv()` function to read the movie_review_*.csv files into 3 separate pandas dataframes

# Note: All the dataframes would have different column names. For testing purposes
# you should have the following column names/headers -> [Title, Year, Synopsis, Review]

def preprocess_data() -> pd.DataFrame:
    """
    Reads movie data from .csv files, map column names, add the "Original Language" column,
    and finally concatenate in one resultant dataframe called "df".
    """
    df_eng = pd.read_csv("data/movie_reviews_eng.csv")
    df_eng.rename(columns={'Movie / TV Series':'Title'}, inplace=True)
    df_eng.insert(4, 'Original Language','English')
    
    #TODO 1: Add your code here
    df_fr = pd.read_csv("data/movie_reviews_fr.csv")
    df_fr.rename(columns={'Titre':'Title','Année':'Year','Critiques':'Review'}, inplace=True)
    df_fr.insert(4, 'Original Language','French')
    
    df_sp = pd.read_csv("data/movie_reviews_sp.csv")
    df_sp.rename(columns={'Título':'Title','Año':'Year','Sinopsis': 'Synopsis', 'Críticas':'Review'}, inplace=True)
    df_sp.insert(4, 'Original Language','Spanish')
    
    frames = [df_eng, df_fr, df_sp]
    df = pd.concat(frames, axis=0, ignore_index = True)
    
    return df

df = preprocess_data()

In [22]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language
0,The Sh...,1994,Andy D...,"""The S...",English
1,The Da...,2008,Batman...,"""The D...",English
2,Forres...,1994,Forres...,"""Forre...",English
3,The Go...,1972,Don Vi...,"""The G...",English
4,Inception,2010,Dom Co...,"""Incep...",English
5,Blade ...,2017,Office...,"""Borin...",English
6,Scott ...,2010,Scott ...,"""It wa...",English
7,The Ni...,2016,In 197...,"""The N...",English
8,Solo: ...,2018,A youn...,"""Dull ...",English
9,The Is...,2005,In a f...,"""The I...",English


### Text translation

Translate the **Review** and **Synopsis** column values to English.

In [23]:
# load translation models and tokenizers
# TODO 2: Update the code below
fr_en_model_name = 'Helsinki-NLP/opus-mt-ROMANCE-en'
es_en_model_name = 'Helsinki-NLP/opus-mt-ROMANCE-en'
fr_en_model = MarianMTModel.from_pretrained(fr_en_model_name)
es_en_model = MarianMTModel.from_pretrained(es_en_model_name)
fr_en_tokenizer = MarianTokenizer.from_pretrained(fr_en_model_name)
es_en_tokenizer = MarianTokenizer.from_pretrained(es_en_model_name)

# TODO 3: Complete the function below
def translate(text: str, model, tokenizer) -> str:
    """
    function to translate a text using a model and tokenizer
    """
    # encode the text using the tokenizer
    inputs = tokenizer(text, return_tensors="pt", padding=True)

    # generate the translation using the model
    outputs = model.generate(**inputs)

    # decode the generated output and return the translated text
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded

/opt/venv/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [25]:
# TODO 4: Update the code below

# filter reviews in French and translate to English
fr_reviews = df[df['Original Language']== 'French']['Review']
fr_reviews_en = [translate(review, fr_en_model, fr_en_tokenizer) for review in fr_reviews]

/opt/venv/lib/python3.10/site-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [26]:
# filter synopsis in French and translate to English
fr_synopsis = df[df['Original Language']== 'French']['Synopsis']
fr_synopsis_en = [translate(synopsis, fr_en_model, fr_en_tokenizer) for synopsis in fr_synopsis]

In [27]:
# filter reviews in Spanish and translate to English
es_reviews = df[df['Original Language']== 'Spanish']['Review']
es_reviews_en = [translate(review, es_en_model, es_en_tokenizer) for review in es_reviews]

In [28]:
# filter synopsis in Spanish and translate to English
es_synopsis = df[df['Original Language']== 'Spanish']['Synopsis']
es_synopsis_en = [translate(synopsis, es_en_model, es_en_tokenizer) for synopsis in es_synopsis]

# update dataframe with translated text
# add the translated reviews and synopsis - you can overwrite the existing data
df.loc[df['Original Language']== 'French','Review'] = fr_reviews_en
df.loc[df['Original Language']== 'French','Synopsis'] = fr_synopsis_en
df.loc[df['Original Language']== 'Spanish','Review'] = es_reviews_en
df.loc[df['Original Language']== 'Spanish','Synopsis'] = es_synopsis_en

### Sentiment Analysis

Use HuggingFace pretrained model for sentiment analysis of the reviews. Store the sentiment result **Positive** or **Negative** in a new column titled **Sentiment** in the dataframe.

In [31]:
# TODO 5: Update the code below
# load sentiment analysis model
model_name = 'sentiment-analysis'
sentiment_classifier = pipeline(task=model_name)


# TODO 6: Complete the function below
def analyze_sentiment(text, classifier):
    """
    function to perform sentiment analysis on a text using a model
    """
    preds = classifier(text)
    return preds[0]["label"]

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [32]:
# TODO 7: Add code below for sentiment analysis
# perform sentiment analysis on reviews and store results in new column
eng_reviews = df[df['Original Language']== 'English']['Review']
en_sentiments = [analyze_sentiment(review, sentiment_classifier) for review in eng_reviews]
fr_sentiments = [analyze_sentiment(review, sentiment_classifier) for review in fr_reviews_en]
es_sentiments = [analyze_sentiment(review, sentiment_classifier) for review in fr_reviews_en]

df.loc[df['Original Language']== 'French','Sentiment'] = fr_sentiments
df.loc[df['Original Language']== 'English','Sentiment'] = en_sentiments
df.loc[df['Original Language']== 'Spanish','Sentiment'] = es_sentiments

In [33]:
df.sample(10)
print(df)

['POSITIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE', 'NEGATIVE', 'NEGATIVE', 'NEGATIVE', 'NEGATIVE', 'NEGATIVE']
        Title       Year   Synopsis     Review Original Language Sentiment
0   The Sh...       1994  Andy D...  "The S...    English         POSITIVE
1   The Da...       2008  Batman...  "The D...    English         POSITIVE
2   Forres...       1994  Forres...  "Forre...    English         POSITIVE
3   The Go...       1972  Don Vi...  "The G...    English         POSITIVE
4   Inception       2010  Dom Co...  "Incep...    English         POSITIVE
5   Blade ...       2017  Office...  "Borin...    English         NEGATIVE
6   Scott ...       2010  Scott ...  "It wa...    English         NEGATIVE
7   The Ni...       2016  In 197...  "The N...    English         NEGATIVE
8   Solo: ...       2018  A youn...  "Dull ...    English         NEGATIVE
9   The Is...       2005  In a f...  "The I...    English         NEGATIVE
10  La La ...       2016  This m...  "La La...     Fre

In [34]:
# export the results to a .csv file
df.to_csv("result/reviews_with_sentiment.csv", index=False)